In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [2]:
llm_model = 'gpt-4'

In [3]:
from langchain_openai import ChatOpenAI

In [4]:
llm = ChatOpenAI(temperature=0.9, model=llm_model, api_key = os.environ['OPENAI_API_KEY'])

### LangChain: Q&A over Documents

In [5]:
!pip install langchain-community docarray

In [21]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.llms import OpenAI

In [7]:
file = 'dset/outdoor_cloth_catalog.csv'
loader = CSVLoader(file_path=file)

In [8]:
from langchain.indexes import VectorstoreIndexCreator

In [12]:
from langchain_openai.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [17]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings
).from_loaders([loader])

/home/agastya/mambaforge/lib/python3.10/site-packages/pydantic/_migration.py:276: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [18]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

In [22]:
llm_replacement_model = ChatOpenAI(temperature=0,
                               model=llm_model)
response = index.query(query,
                       llm = llm_replacement_model)

In [23]:
display(Markdown(response))

Sure, here are the shirts with sun protection:

| Product ID | Name | Description | Material | Sun Protection | Additional Features |
| --- | --- | --- | --- | --- | --- |
| 618 | Men's Tropical Plaid Short-Sleeve Shirt | A light hot-weather shirt with a traditional fit. It is wrinkle-resistant and has front and back cape venting. | 100% Polyester | UPF 50+, blocks 98% of UV rays | Two front bellows pockets |
| 374 | Men's Plaid Tropic Shirt, Short-Sleeve | A light hot-weather shirt originally designed for fishing. It is wrinkle-free and quickly evaporates perspiration. | 52% Polyester, 48% Nylon | UPF 50+, blocks 98% of UV rays | Front and back cape venting, two front bellows pockets |
| 255 | Sun Shield Shirt | A high-performance sun shirt that wicks moisture for quick-drying comfort. It is abrasion resistant. | 78% Nylon, 22% Lycra Xtra Life fiber | UPF 50+, blocks 98% of UV rays | Fits comfortably over your favorite swimsuit |
| 535 | Men's TropicVibe Shirt, Short-Sleeve | A sun-protection shirt with built-in UPF 50+ that has a lightweight feel. It is wrinkle resistant. | Shell: 71% Nylon, 29% Polyester. Lining: 100% Polyester knit mesh | UPF 50+, blocks 98% of UV rays | Front and back cape venting, two front bellows pockets |

### Step by Step

In [37]:
from langchain.document_loaders import CSVLoader
file = 'dset/outdoor_cloth_catalog-limited.csv'
loader = CSVLoader(file_path = file)

In [38]:
docs = loader.load()
docs[0]

Document(metadata={'source': 'dset/outdoor_cloth_catalog-limited.csv', 'row': 0}, page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \r\n\r\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \r\n\r\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \r\n\r\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \r\n\r\nQuestions? Please contact us for any inquiries.")

In [39]:
embed = embeddings.embed_query("Hi my name is harrison")

In [40]:
print(len(embed), embed[:5])

1536 [-0.022224984768242112, 0.00861565871347167, -0.02159287570911561, -0.03651066887600874, -0.016396932574826772]


In [41]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)

In [42]:
query = "Please suggest a shirt with sunblocking"

In [43]:
docs = db.similarity_search(query)

In [44]:
docs

[Document(metadata={'source': 'dset/outdoor_cloth_catalog-limited.csv', 'row': 255}, page_content=': 255\nname: Sun Shield Shirt by\ndescription: "Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. \r\n\r\nSize & Fit: Slightly Fitted: Softly shapes the body. Falls at hip.\r\n\r\nFabric & Care: 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated – the highest rated sun protection possible. Handwash, line dry.\r\n\r\nAdditional Features: Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion resistant for season after season of wear. Imported.\r\n\r\nSun Protection That Won\'t Wear Off\r\nOur high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun\'s harmful rays. This fabric is recommended by The Skin Cancer Foundation as an effective UV protectant.'),
 Document(metadata={'source': 'dset/outdoor_cloth_catalog-limited.csv', 'row': 374}, page_content=": 374\nname: Men'

In [46]:
len(docs), docs[0]

(4,
 Document(metadata={'source': 'dset/outdoor_cloth_catalog-limited.csv', 'row': 255}, page_content=': 255\nname: Sun Shield Shirt by\ndescription: "Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. \r\n\r\nSize & Fit: Slightly Fitted: Softly shapes the body. Falls at hip.\r\n\r\nFabric & Care: 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated – the highest rated sun protection possible. Handwash, line dry.\r\n\r\nAdditional Features: Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion resistant for season after season of wear. Imported.\r\n\r\nSun Protection That Won\'t Wear Off\r\nOur high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun\'s harmful rays. This fabric is recommended by The Skin Cancer Foundation as an effective UV protectant.'))

In [48]:
retriever = db.as_retriever()
retriever

VectorStoreRetriever(tags=['DocArrayInMemorySearch'], vectorstore=<langchain_community.vectorstores.docarray.in_memory.DocArrayInMemorySearch object at 0x7f587619c430>)

In [49]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])

In [50]:
qdocs

': 255\nname: Sun Shield Shirt by\ndescription: "Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. \r\n\r\nSize & Fit: Slightly Fitted: Softly shapes the body. Falls at hip.\r\n\r\nFabric & Care: 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated – the highest rated sun protection possible. Handwash, line dry.\r\n\r\nAdditional Features: Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion resistant for season after season of wear. Imported.\r\n\r\nSun Protection That Won\'t Wear Off\r\nOur high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun\'s harmful rays. This fabric is recommended by The Skin Cancer Foundation as an effective UV protectant.: 374\nname: Men\'s Plaid Tropic Shirt, Short-Sleeve\ndescription: Our Ultracomfortable sun protection is rated to UPF 50+, helping you stay cool and dry. Originally designed for fishing, this lightest hot-weather shir

In [51]:
response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.")

/home/agastya/mambaforge/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.call_as_llm` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [54]:
display(Markdown(response))

| Shirt Name | Material Composition | Sun Protection Rating | Additional Features |
|------------|------------------------|-------------------------|---------------------|
| Sun Shield Shirt| 78% nylon, 22% Lycra Xtra Life fiber | UPF 50+ | Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion resistant for season after season of wear. |
| Men's Plaid Tropic Shirt, Short-Sleeve | 52% polyester and 48% nylon | UPF 50+ | Front and back cape venting, two front bellows pockets, wrinkle-free and quickly evaporates perspiration. |
| Men's TropicVibe Shirt, Short-Sleeve | Shell: 71% Nylon, 29% Polyester. Lining: 100% Polyester knit mesh | UPF 50+ | Wrinkle resistant. Front and back cape venting lets in cool breezes. Two front bellows pockets. |
| Men's Tropical Plaid Short-Sleeve Shirt | 100% polyester | UPF 50+ | Front and back cape venting that lets in cool breezes and two front bellows pockets. Wrinkle-resistant. |

Summary:

1. Sun Shield Shirt: This shirt not only blocks harmful UV rays, but also wicks moisture for quick drying. It fits comfortably over any swimsuit and is abrasion-resistant for durable wear.
2. Men's Plaid Tropic Shirt, Short-Sleeve: Originally designed for fishing, this shirt provides excellent sun protection and quickly evaporates perspiration. It also features front and back cape venting and two front bellows pockets.
3. Men's TropicVibe Shirt, Short-Sleeve: This shirt has built-in sun protection and a lightweight feel. It is wrinkle-resistant with front and back cape venting for cool breezes and two front bellows pockets for added convenience.
4. Men's Tropical Plaid Short-Sleeve Shirt: This shirt provides superior sun protection. It includes front and back cape venting to let in cool breezes, two front bellows pockets for convenience, and is wrinkle-resistant for a polished look.

In [55]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [56]:
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."

In [57]:
response = qa_stuff.run(query)

/home/agastya/mambaforge/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...

> Finished chain.


In [62]:
Markdown(response)

Sure, here's the list of shirts with sun protection:

|   ID  |                    Name                    |                                               Summary                                               |
|:-----:|:----------------------------------------:|:-------------------------------------------------------------------------------------------------:|
|  618  |  Men's Tropical Plaid Short-Sleeve Shirt  |  100% polyester shirt, UPF 50+ rated, wrinkle-resistant, includes venting, two pockets, imported.  |
|  374  |     Men's Plaid Tropic Shirt, Short-Sleeve    |  52% polyester and 48% nylon shirt, UPF 50+ rated, wrinkle-resistant, evaporates perspiration, includes venting, two pockets, machine washable, imported.  |
|  255  |             Sun Shield Shirt by           |  78% nylon, and 22% Lycra shirt, UPF 50+ rated, recommended by The Skin Cancer Foundation, handwash, line dry. |
|  535  | Men's TropicVibe Shirt, Short-Sleeve      | 71% nylon, and 29% polyester shirt, UPF 50+ rated, wrinkle-resistant, includes venting, two pockets, machine washable, imported. |

In [63]:
response = index.query(query, llm=llm)